In [ ]:
import argparse
import os
import random
import shutil
import time
import warnings
from enum import Enum
import pickle
import numpy as np
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
import torch.backends.cudnn as cudnn

def my_norm(x):
    return x/np.linalg.norm(x, axis=-1, keepdims=True)

In [ ]:
pickle_path = './features/feature_dump_val.pkl'
with open(pickle_path, 'rb') as pkl_file:
    data_dict = pickle.load(pkl_file)
    assert len(data_dict['clip_image_features_list']) == len(data_dict['clip_text_features_list'])
    # assert len(data_dict['clip_image_features_list']) == len(data_dict['target_image_features_list'])
    print('Number of image-text pairs', len(data_dict['clip_image_features_list']))

In [ ]:
# Get the gap
modality_gap = my_norm(my_norm(data_dict['clip_image_features_list']).mean(axis=0) - my_norm(data_dict['clip_text_features_list']).mean(axis=0))
# # save as a gap vector
# with open(os.path.join('modality_gap_vector.pkl' ), "wb") as pkl_file:
#     pickle.dump(
#         modality_gap, 
#         pkl_file, 
#     )

In [20]:
# modifying_results = defaultdict(list)

# for delta in np.arange(-1.0, 1.0, 0.25): 
#     modified_text_features = my_norm(data_dict['clip_text_features_list']) + 0.5 * delta * modality_gap
#     modified_text_features = my_norm(modified_text_features)

#     modified_image_features = my_norm(data_dict['clip_image_features_list']) - 0.5 * delta * modality_gap
#     modified_image_features = my_norm(modified_image_features)

#     distance_sign = np.dot(modality_gap, modified_image_features.mean(axis=0)-modified_text_features.mean(axis=0))
    
#     # Euclidean distance between mass centers
#     modifying_results['distance'].append(
#         np.linalg.norm(
#             modified_image_features.mean(axis=0) - modified_text_features.mean(axis=0)
#         ) * np.sign(distance_sign)
#     )
#     modifying_results['delta'].append(delta)

# import pandas as pd
# print('modifying_results')

# pd.DataFrame(modifying_results)

modifying_results


,distance,delta
0,1.412827,-1.00
1,1.305014,-0.75
2,1.173114,-0.50
3,1.013107,-0.25
4,0.822103,0.00
5,0.599846,0.25
6,0.350383,0.50
7,0.083829,0.75


In [23]:
modifying_results = defaultdict(list)

for delta in np.arange(-1.0, 1.0, 0.05): 
    modified_text_features = my_norm(data_dict['clip_text_features_list']) 
    modified_text_features = my_norm(modified_text_features)

    modified_image_features = my_norm(data_dict['clip_image_features_list']) - 1.0 * delta * modality_gap
    modified_image_features = my_norm(modified_image_features)

    distance_sign = np.dot(modality_gap, modified_image_features.mean(axis=0)-modified_text_features.mean(axis=0))
    
    # Euclidean distance between mass centers
    modifying_results['distance'].append(
        np.linalg.norm(
            modified_image_features.mean(axis=0) - modified_text_features.mean(axis=0)
        ) * np.sign(distance_sign)
    )
    modifying_results['delta'].append(delta)

import pandas as pd
print('modifying_results')

pd.DataFrame(modifying_results)


modifying_results


,distance,delta
0,1.287905,-1.00
1,1.224927,-0.75
2,1.134930,-0.50
3,1.005094,-0.25
4,0.822103,0.00
5,0.584231,0.25
6,0.317848,0.50
7,0.070293,0.75


In [ ]:
# save as a gap vector
dump_dict = {
    'modality_gap': modality_gap, 
    'Eucliean distance': modifying_results['distance'], 
    'delta': modifying_results['delta'],
}
with open(os.path.join('modality_gap_vector.pkl' ), "wb") as pkl_file:
    pickle.dump(
        dump_dict, 
        pkl_file, 
    )